# Interactive map with binder

Interactive map with binder:
All detail and credit go to: https://walkenho.github.io/beergarden-happiness-with-python/

## Setup imports

In [1]:
import pandas as pd
import requests
import wget

import folium
from folium.plugins import MarkerCluster
import os
import time


## Getting data
Use wget to download the file and read it into a pandas data frame. Make sure to set the encoding since the file contains special characters (lots of Cafes on the list).

In [2]:
filename = wget.download('http://www.edinburgh.gov.uk/download/downloads/id/11854/tables_and_chairs_permits.csv')
df0 = pd.read_csv(filename, encoding = 'ISO-8859-1')

df0.head()
